Data Source

https://www.kaggle.com/datasets/luisblanche/covidct

In [ ]:
# pip install tensorflow==2.13.0

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [ ]:
tf.executing_eagerly()

In [ ]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
# import tensorflow.keras.backend as K
#tf.keras.backend.clear_session()
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    print(gpu)


In [ ]:
import tensorflow as tf
def limitgpu(maxmem):
	gpus = tf.config.list_physical_devices('GPU')
	if gpus:
		# Restrict TensorFlow to only allocate a fraction of GPU memory
		try:
			for gpu in gpus:
				tf.config.experimental.set_virtual_device_configuration(gpu,
						[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=maxmem)])
		except RuntimeError as e:
			# Virtual devices must be set before GPUs have been initialized
			print(e)


# 1.5GB
limitgpu(1024+512) 


In [ ]:
pip show tensorflow

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Create 2 virtual GPUs with 1GB memory each
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024),
         tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [ ]:
!pip install opencv-python==3.4.18.65

In [ ]:
## Importing Relevant Libraries

In [ ]:
import pandas as pd
import cv2
import os.path
from PIL import Image
from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
import os, logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  
from os import listdir
import glob
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
#import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow import keras
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"
a = tf.zeros([], tf.float32)
## Imports libs

In [ ]:
print("OpenCV version:", cv2.__version__)

In [ ]:
pip install lime==0.1.1.29

In [ ]:
pip show lime

## Renaming images data to Specified format

In [ ]:
def renameImageFiles(folderpath, prefix,fileExtension):
    folder_path = folderpath
    new_prefix = prefix

    for i, file_path in enumerate(glob.glob(folder_path + '*.'+fileExtension)):
        new_file_name = new_prefix + '_' + str(i+1) + '.'+fileExtension
        os.rename(file_path, os.path.join(folder_path, new_file_name))


In [ ]:
!pwd

In [ ]:
path1 = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_merged/CT_COVID/'
prefix1 = 'ct_covid'
path2 = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_merged/CT_NonCOVID/'
prefix2 = 'ct_noncovid'

# renameImageFiles(path1, prefix1,'png')
# renameImageFiles(path2, prefix2,'png')
# renameImageFiles(path2, prefix2,'jpg')

## Data Preprocessing
### Processing images for machine learning to create two-class dataset

In [ ]:
def processImages(imgDirPath, binary_label):
    img_names = list()
    try:
        with os.scandir(imgDirPath) as dirs:
            for entry in dirs:
                img_names.append(entry.name)
    except Exception as e:
        print(f"Error while scanning directory: {e}")
        return None

    all_features = []
    for img in img_names:
        try:
            path = imgDirPath + img
            cv_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if cv_img is None:
                print(f"Error reading image: {path}")
                continue

            cv_img2 = cv2.resize(cv_img, (300, 300), interpolation=cv2.INTER_NEAREST)

            nFeatures = (cv_img2.shape[0] * cv_img2.shape[1])
            features = np.reshape(cv_img2, nFeatures)
            all_features.append(features)
        except Exception as e:
            print(f"Error processing image {img}: {e}")

    if len(all_features) == 0:
        print("No valid images found.")
        return None

    imgs_df = pd.DataFrame(np.array(all_features), index=img_names)
    if binary_label == 0:
        imgs_df['class_label'] = np.zeros((imgs_df.shape[0]), dtype=int)
    else:
        imgs_df['class_label'] = np.ones((imgs_df.shape[0]), dtype=int)

    return imgs_df

#### Processing CT_scan Covid-19  CT_scan nonCovid-19 images

In [ ]:
ct_covid_features_df =  processImages(path1,1)#1--> covid-19 positive
ct_noncovid_features_df =  processImages(path2,0)#0 ---> covnid-19 negative

In [ ]:
ct_covid_features_df

In [ ]:
#imshow(ct_covid_features_df.iloc[0].to_numpy().reshape((100,150)))

In [ ]:
ct_noncovid_features_df

In [ ]:
#imshow(ct_noncovid_features_df.iloc[1].to_numpy().reshape((100,150)))

#### Concatenating the two data frame to create a dataset ready for machine learning

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset)

In [ ]:
ct_covid_features_df.shape

In [ ]:
ct_noncovid_features_df.shape

In [ ]:
cvd_imgs_dataset.shape

In [ ]:
#### Prepare negative covid images for machine learning ready

In [ ]:
input_data_x_ = ct_noncovid_features_df.iloc[:,:-1].to_numpy().reshape((2173,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_noncovid_features_df['output_encode'] = label_encoder.fit_transform(ct_noncovid_features_df['class_label'])
ct_noncovid_features_df
ct_noncovid_features_df = pd.get_dummies(ct_noncovid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y_ = np.array(ct_noncovid_features_df[['output_encode_0']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x_.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y_.shape))

In [ ]:
#### Prepare positive covid images for machine learning ready

In [ ]:
input_data_x__ = ct_covid_features_df.iloc[:,:-1].to_numpy().reshape((2476,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
ct_covid_features_df['output_encode'] = label_encoder.fit_transform(ct_covid_features_df['class_label'])
ct_covid_features_df
ct_covid_features_df = pd.get_dummies(ct_covid_features_df, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y__ = np.array(ct_covid_features_df[['output_encode_0']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x__.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y__.shape))

In [ ]:
input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy().reshape((4649,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset['class_label'])
cvd_imgs_dataset
cvd_imgs_dataset = pd.get_dummies(cvd_imgs_dataset, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_y = np.array(cvd_imgs_dataset[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_y.shape))

In [ ]:
type(input_data_x)

In [ ]:
input_data_x.shape

In [ ]:
output_label_y.shape

In [ ]:
## Images of Negative Covid-19
for i in range(10):
    imshow(input_data_x_[i])
    show()
    

In [ ]:
## Images of Positive Covid-19
for i in range(15):
    imshow(input_data_x__[i])
    show()
    

#### visualize first 5 images from the dataset

In [ ]:
for i in range(15):
    imshow(input_data_x[i])
    show()
    

### Feature Scaling

#### **Splitting the dataset into the Training set and Test set**

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(
    input_data_x, output_label_y, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features_, test_features_, train_labels_, test_labels_ = train_test_split(
    input_data_x_, output_label_y_, test_size=.20, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

train_features__, test_features__, train_labels__, test_labels__ = train_test_split(
    input_data_x__, output_label_y__, test_size=.20, random_state=42)

In [ ]:
train_features__.shape

In [ ]:
test_features__.shape

### IMPLEMENTATION OF SHAP

In [ ]:
import shap
from shap import initjs
initjs()
custom_objects = None
#model_mse = tf.keras.models.load_model('VGG19_model.h5', custom_objects=custom_objects, compile=False)

In [ ]:
model_mse.summary()

In [ ]:
###VGG19

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense

In [ ]:
def VGG19(input_shape=None, classes=5, use_soft=True):
    img_input = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(2, (3, 3), activation='relu', padding='same', name='block1_conv1', kernel_initializer="he_normal")(img_input)
    x = layers.Conv2D(2, (3, 3), activation='relu', padding='same', name='block1_conv2', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = layers.Conv2D(4, (3, 3), activation='relu', padding='same', name='block2_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(4, (3, 3), activation='relu', padding='same', name='block2_conv2', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv2', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv3', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', name='block3_conv4', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv2', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv3', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv4', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv1', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv2', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv3', kernel_initializer="he_normal")(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv4', kernel_initializer="he_normal")(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    # Classification block
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dense(512, activation='relu', name='fc1')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu', name='fc2')(x)

    # Output layer
    if use_soft:
        x = Dense(classes, activation="softmax", name='predictions')(x)
    else:
        x = Dense(classes, activation="linear", name="Z_4")(x)

    modelss = models.Model(img_input, x, name='vgg19')

    return modelss


In [ ]:
modelss = VGG19(input_shape=(300, 300, 1), classes=2)
modelss.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
train_features.shape

In [ ]:
train_labels.shape

In [ ]:
modelss.fit(train_features, train_labels, epochs=20, validation_data=(test_features, test_labels))

In [ ]:
modelss.save('VGG19_model.h5')

In [ ]:
###Explainer for VGG19 Model

In [ ]:
model_VGG19 = tf.keras.models.load_model('VGG19_model.h5')

In [ ]:
model_VGG19.summary()

In [ ]:
masker = shap.maskers.Image("blur(32,32)", test_features.shape)

In [ ]:
masked_images_as_lists = []
image_shape = (300, 300, 1)

for image in test_features:
    # Ensure the image has the correct shape
    if image.shape != image_shape:
        raise ValueError(f"Image shape {image.shape} does not match expected shape {image_shape}")

    # Initialize the masker for a single image
    masker = shap.maskers.Image("blur(32,32)", image_shape)

    # Apply the masker to each image
    masked_image_tuple = masker(image, np.zeros(image.shape[:-1]))  # Adjust the mask as needed
    
    # Extract the first element of the tuple (which is the masked image)
    masked_image = masked_image_tuple[0]
    
    # Convert the masked image to a list and store it
    masked_images_as_lists.append(masked_image.tolist())


In [ ]:
## SHAP for Positve and Negative

In [ ]:
explainVGG19 = shap.GradientExplainer(model_VGG19,train_features)
explainVGG19 

In [ ]:
shap_values_ = explainVGG19.shap_values(test_features)

In [ ]:
shap_values_.shape

In [ ]:
shap.image_plot(shap_values_[:5], test_features[:5])

In [ ]:
### Explainer and SHAP for Negative

In [ ]:
explainVGG19_negative = shap.GradientExplainer(model_VGG19,train_features_) 

In [ ]:
shap_values_negative = explainVGG19_negative.shap_values(test_features_)

In [ ]:
shap.image_plot(shap_values_negative[:5], test_features_[:5])

In [ ]:
### Explainer and SHAP for Positive

In [ ]:
explainVGG19_positive = shap.GradientExplainer(model_VGG19,train_features__) 

In [ ]:
shap_values_positive = explainVGG19_positive.shap_values(test_features__)

In [ ]:
shap.image_plot(shap_values_positive[:5], test_features_[:5])

In [ ]:
###Mask out the Important regions

In [ ]:
import numpy as np
import cv2
import os

import numpy as np
import cv2
import os

def process_and_save_images(images, shap_values, threshold, output_folder):
    """
    Process images by removing non-important features based on SHAP values,
    change the background to white, and apply a red color to non-important features.

    Parameters:
    - images: numpy array of shape (num_images, height, width, channels)
    - shap_values: numpy array of shape (num_images, height, width, channels)
    - threshold: importance threshold to decide which features are important
    - output_folder: directory to save the processed images
    """

    # Create masks based on SHAP values and threshold
    important_features_mask = np.abs(shap_values) > threshold

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process and save each image
    for idx, (img, mask) in enumerate(zip(images, important_features_mask)):
        # Ensure image and mask are of the same shape
        if img.shape != mask.shape:
            raise ValueError(f"Image shape {img.shape} and mask shape {mask.shape} do not match.")
        
        # Create a white background
        white_background = np.ones_like(img) * 255

        # Apply mask to keep important features and set the rest to white
        processed_img = np.where(mask, img, white_background)

        # Convert non-important features (where mask is False) to red (for RGB images)
        if img.shape[-1] == 3:  # If the image has 3 channels (colored image)
            red_mask = np.zeros_like(img)
            red_mask[..., 2] = 255  # Red color in RGB

            # Ensure to only modify non-important features (where mask is False)
            processed_img[~mask] = red_mask[~mask]

        # Ensure the image is within the range 0-255 for saving
        processed_img = processed_img.astype(np.uint8)

        # Save the image
        output_file = os.path.join(output_folder, f'processed_image_{idx}.png')
        cv2.imwrite(output_file, processed_img)


In [ ]:
##### Images for only negative images

In [ ]:
shap_negative = shap_values_negative[:, :, :, :, 0]
shap_negative.shape

In [ ]:
threshold = np.percentile(np.abs(shap_values_negative), 95)
save_path = "dataset_important_features/VGG19/SHAP/Negative" 
process_and_save_images(test_features_, shap_negative, threshold, save_path)

In [ ]:
#### Images for only positive Images

In [ ]:
shap_positive = shap_values_positive[:, :, :, :, 0]
shap_positive.shape

In [ ]:
threshold = np.percentile(np.abs(shap_values_positive), 95)
save_path = "dataset_important_features/VGG19/SHAP/Positive" 
process_and_save_images(test_features__, shap_negative, threshold, save_path)

In [ ]:
## Retrain

In [ ]:
new_path_ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/SHAP/Positive/'
new_prefix = 'ct_covid'

new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/SHAP/Negative/'
new_prefix_ = 'non_ct_covid'

In [ ]:
ct_covid_features_df_ =  processImages(new_path_,1)#1--> covid-19 positive
ct_noncovid_features_df_ =  processImages(new_path__,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs_ = [ct_noncovid_features_df_, ct_covid_features_df_]
cvd_imgs_dataset_ = pd.concat(cvd_imgs_)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_shap_ = cvd_imgs_dataset_.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_shap_)

In [ ]:
cvd_imgs_dataset_shap_.shape

In [ ]:
input_data_shap_x_ = cvd_imgs_dataset_shap_.iloc[:,:-1].to_numpy().reshape((870,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_shap_['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_shap_['class_label'])
cvd_imgs_dataset_shap_
cvd_imgs_dataset_shap_ = pd.get_dummies(cvd_imgs_dataset_shap_, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_shap_y_ = np.array(cvd_imgs_dataset_shap_[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_shap_x_.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_shap_y_.shape))

In [ ]:
for i in range(5):
    imshow(input_data_shap_x_[i])
    show()

In [ ]:
from sklearn.model_selection import train_test_split

train_features_shap_, test_features_shap_, train_labels_shap_, test_labels_shap_ = train_test_split(
    input_data_shap_x_, output_label_shap_y_, test_size=.20, random_state=42)

In [ ]:
modelss.fit(train_features_shap_, train_labels_shap_, epochs=20, validation_data=(test_features, test_labels))

In [ ]:
my_model_shap = shap_values_[:, :, :, :, 0]
my_model_shap.shape

In [ ]:
threshold = np.percentile(np.abs(shap_values_), 95)
save_path = "dataset_important_features/VGG19/SHAP"  # Replace with your directory path
process_and_save_images(test_features, my_model_shap, threshold, save_path)

In [ ]:
###Retrain the model on important images only

In [ ]:
new_path = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/SHAP/'
new_prefix = 'ct_covid'

In [ ]:
ct_covid_shap_features_VGG19_df =  processImages(new_path,1)
ct_covid_shap_features_VGG19_df

In [ ]:
cvd_imgs = [ct_noncovid_features_df, ct_covid_shap_features_VGG19_df]
cvd_imgs_dataset = pd.concat(cvd_imgs)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_shap_VGG19 = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_shap_VGG19)

In [ ]:
cvd_imgs_dataset_shap_VGG19.shape

In [ ]:
input_data_shap_VGG19_x = cvd_imgs_dataset_shap_VGG19.iloc[:,:-1].to_numpy().reshape((3103,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_shap_VGG19['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_shap_VGG19['class_label'])
cvd_imgs_dataset_shap_VGG19
cvd_imgs_dataset_shap_VGG19 = pd.get_dummies(cvd_imgs_dataset_shap_VGG19, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_shap_VGG19_y = np.array(cvd_imgs_dataset_shap_VGG19[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_shap_VGG19_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_shap_VGG19_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_shap_VGG19, test_features_shap_VGG19, train_labels_shap_VGG19, test_labels_shap_VGG19 = train_test_split(
    input_data_shap_VGG19_x, output_label_shap_VGG19_y, test_size=.20, random_state=42)

In [ ]:
train_features_shap_VGG19.shape

In [ ]:
model_vgg19 = VGG19(input_shape=(300, 300, 1), classes=2)
model_vgg19.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model_vgg19.fit(train_features_shap_VGG19, train_labels_shap_VGG19, epochs=10, validation_data=(test_features_shap_VGG19, test_labels_shap_VGG19))

In [ ]:
###Apply LIME

In [ ]:
train_features[0].shape

In [ ]:
### LIME on negative test features

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries, felzenszwalb

# Function to predict on the model
def predict_fn(images):
    return model_VGG19.predict(images)

single_image = test_features_[0] 
single_image.shape[0]
num_images_to_explain = 300
explainer = lime_image.LimeImageExplainer()
explanations = []
masks = []
segmentation_fn = lambda x: felzenszwalb(x, scale=100, sigma=0.5, min_size=50)

for i in range(num_images_to_explain):
    single_image = test_features_[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation = explainer.explain_instance(single_image,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations.append(explanation)
    
    # Get explanation for the top predicted class
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], 
                                                positive_only=True, 
                                                num_features=10, 
                                                hide_rest=False)
    masks.append(mask)

     # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(test_features[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')
    plt.show()

    
    # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(mark_boundaries(single_image.squeeze(), mask), cmap='gray')
    plt.title(f"Explanation for Image {i+1}")
    plt.axis('off')
    plt.show()




In [ ]:
######## LIME for postive test sample 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries, felzenszwalb

# Function to predict on the model
def predict_fn(images):
    return model_VGG19.predict(images)

single_image_ = test_features__[0] 
single_image_.shape[0]
num_images_to_explain = 300
explainer = lime_image.LimeImageExplainer()
explanations = []
masks = []
segmentation_fn = lambda x: felzenszwalb(x, scale=100, sigma=0.5, min_size=50)

for i in range(num_images_to_explain):
    single_image = test_features_[i]  # Select the i-th image in the batch
    
    # Explain the prediction for the grayscale image
    explanation = explainer.explain_instance(single_image_,
                                             predict_fn, 
                                             top_labels=10, 
                                             hide_color=0, 
                                             num_samples=1000,
                                             segmentation_fn=segmentation_fn)
    
    explanations.append(explanation)
    
    # Get explanation for the top predicted class
    temp, mask_VGG16 = explanation.get_image_and_mask(explanation.top_labels[0], 
                                                positive_only=True, 
                                                num_features=10, 
                                                hide_rest=False)
    masks.append(mask)

     # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(test_features[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')
    plt.show()

    
    # Show the explanation
    plt.figure(figsize=(5, 5))
    plt.imshow(mark_boundaries(single_image.squeeze(), mask), cmap='gray')
    plt.title(f"Explanation for Image {i+1}")
    plt.axis('off')
    plt.show()




In [ ]:
masks_VGG19 = np.array(masks)

In [ ]:
import numpy as np
from skimage.io import imsave
from skimage.util import img_as_ubyte
from skimage.filters import gaussian
import os

def apply_lime_mask(images, lime_masks, method='blur', blur_sigma=5, save=False, save_path='dataset_lime_filtered'):
    """
    Removes unimportant regions from images based on LIME masks.

    Parameters:
    - images (list or np.ndarray): List or array of original images.
    - lime_masks (list or np.ndarray): List or array of LIME masks (1 for important regions, 0 for unimportant).
    - method (str): Method to fill unimportant regions ('blur', 'mean', or 'zero').
    - blur_sigma (int): Sigma for Gaussian blur if 'blur' method is selected.
    - save (bool): Whether to save processed images.
    - save_path (str): Directory path to save images if save is True.

    Returns:
    - processed_images (list): List of images with unimportant regions removed.
    """
    
    # Verify input types
    if not isinstance(images, (list, np.ndarray)):
        raise TypeError("Images should be a list or numpy array of image arrays.")
        
    if not isinstance(lime_masks, (list, np.ndarray)):
        raise TypeError("LIME masks should be a list or numpy array of mask arrays.")
        
    if len(images) != len(lime_masks):
        raise ValueError("The number of images and LIME masks must be the same.")
    
    # Create save directory if needed
    if save and not os.path.exists(save_path):
        os.makedirs(save_path)
    
    processed_images = []
    
    for idx, (image, lime_mask) in enumerate(zip(images, lime_masks)):
        # Ensure image and mask dimensions match
        if image.shape[:2] != lime_mask.shape:
            raise ValueError(f"Image and LIME mask at index {idx} have mismatched dimensions.")
        
        # Copy the image for processing
        processed_image = image.copy()
        
        # Apply the chosen method to unimportant regions (where lime_mask == 0)
        if method == 'blur':
            blurred_image = gaussian(processed_image, sigma=blur_sigma)
            processed_image[lime_mask == 0] = blurred_image[lime_mask == 0]
        
        elif method == 'mean':
            mean_value = image.mean(axis=(0, 1), keepdims=True) if image.ndim == 3 else image.mean()
            processed_image[lime_mask == 0] = mean_value
        
        elif method == 'zero':
            processed_image[lime_mask == 0] = 0
        
        else:
            raise ValueError("Invalid method chosen. Available methods: 'blur', 'mean', 'zero'.")
        
        # Append processed image to the list
        processed_images.append(processed_image)
        
        # Save processed image if required
        if save:
            image_to_save = img_as_ubyte(processed_image)
            imsave(os.path.join(save_path, f'lime_filtered_image_{idx}.png'), image_to_save, check_contrast=False)
            print(f"Saved: {os.path.join(save_path, f'lime_filtered_image_{idx}.png')}")
    
    return processed_images


In [ ]:
#### Remove unimportant regions negative

In [ ]:
from skimage.transform import resize

def resize_mask(mask, image_shape):
    return resize(mask, image_shape[:2], mode='reflect', anti_aliasing=True, preserve_range=True)


In [ ]:
# Check dimensions before processing
print(f"Original Image shape: {single_image[0].shape}")
print(f"Original Mask shape: {mask[0].shape}")

# Resize the mask if needed
resized_masks = [resize_mask(mask, image.shape) for mask, image in zip(masks, single_image)]

# Process images with the resized masks
processed_images = apply_lime_mask(
    images=single_image,
    lime_masks=resized_masks,
    method='mean',       # Options: 'mean', 'zero', 'blur'
    blur_sigma=3,        # Used only if method='blur'
    save=True,           # Set to True to save images
    save_path='dataset_important_features/VGG19/LIME/Negative'  # Directory to save images
)


In [ ]:
#### Remove unimportant regions positive 

In [ ]:
# Check dimensions before processing
print(f"Original Image shape: {single_image[0].shape}")
print(f"Original Mask shape: {mask[0].shape}")

# Resize the mask if needed
resized_masks = [resize_mask(mask_VGG16, image.shape) for mask, image in zip(mask_VGG16, single_image)]

# Process images with the resized masks
processed_images = apply_lime_mask(
    images=single_image,
    lime_masks=resized_masks,
    method='mean',       # Options: 'mean', 'zero', 'blur'
    blur_sigma=3,        # Used only if method='blur'
    save=True,           # Set to True to save images
    save_path='dataset_important_features/VGG19/LIME/Positive'  # Directory to save images
)


In [ ]:
### Train on new dataset

In [ ]:
new_path___ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/LIME/Positive/'
new_prefix__ = 'ct_covid'

new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/LIME/Negative/'
new_prefix__ = 'non_ct_covid'

In [ ]:
ct_covid_features_df_lime =  processImages(new_path___,1)#1--> covid-19 positive
ct_noncovid_features_df_lime =  processImages(new_path__,0)#0 ---> covnid-19 negative

In [ ]:
cvd_imgs_lime = [ct_noncovid_features_df_lime, ct_covid_features_df_lime]
cvd_imgs_dataset_lime = pd.concat(cvd_imgs_lime)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_lime_ = cvd_imgs_dataset_lime.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_lime_)

In [ ]:
cvd_imgs_dataset_lime_.shape

In [ ]:
input_data_lime_x = cvd_imgs_dataset_lime_.iloc[:,:-1].to_numpy().reshape((600,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_lime_['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_lime_['class_label'])
cvd_imgs_dataset_lime_
cvd_imgs_dataset_lime_ = pd.get_dummies(cvd_imgs_dataset_lime_, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_lime_y = np.array(cvd_imgs_dataset_lime_[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_lime_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_lime_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_lime, test_features_lime, train_labels_lime, test_labels_lime = train_test_split(
    input_data_lime_x, output_label_lime_y, test_size=.20, random_state=42)

In [ ]:
model_vgg19 = VGG19(input_shape=(300, 300, 1), classes=2)
model_vgg19.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model_vgg19.fit(train_features_lime, train_labels_lime, epochs=100, validation_data=(test_features_lime, test_labels_lime))

In [ ]:
### Intersection

In [ ]:
masker_shap = np.array(masked_images_as_lists)

In [ ]:
masker_shap_ = masker_shap[:, 0, :, :]

In [ ]:
masker_shap_.shape

In [ ]:
test_features_ = test_features[:, :, :, 0]

In [ ]:
test_features_.shape

In [ ]:
test_features_[0].shape

In [ ]:
mask.shape

In [ ]:
new_mask = np.repeat(mask[np.newaxis, :, :], 930, axis=0) 

In [ ]:
new_mask.shape

In [ ]:
import numpy as np
import os
from skimage.io import imsave
from skimage.util import img_as_ubyte

import numpy as np
import os
from skimage.io import imsave
from skimage.util import img_as_ubyte

def find_intersection_and_save(images, shap_masks, lime_masks, save=False, save_path='dataset_intersection'):
    if len(images) != len(shap_masks) or len(images) != len(lime_masks):
        raise ValueError("The number of images, SHAP masks, and LIME masks must be the same.")
    
    if save and not os.path.exists(save_path):
        os.makedirs(save_path)

    intersection_images = []

    for idx, (image, shap_mask, lime_mask) in enumerate(zip(images, shap_masks, lime_masks)):
        if image.shape[:2] != shap_mask.shape or image.shape[:2] != lime_mask.shape:
            raise ValueError(f"Image and masks at index {idx} have mismatched dimensions.")
        
        shap_mask = shap_mask.astype(bool)
        lime_mask = lime_mask.astype(bool)
        
        intersection_mask = np.logical_and(shap_mask, lime_mask)
        print(f"Intersection count at index {idx}: {np.sum(intersection_mask)}")  # Debugging line
        
        intersection_image = np.copy(image)

        if intersection_image.ndim == 2:
            intersection_image = np.stack([intersection_image] * 3, axis=-1)

        overlay_image = np.copy(intersection_image)
        highlight_color = [255, 0, 0]

        overlay_image[intersection_mask] = highlight_color

        final_image = np.where(intersection_mask[:, :, np.newaxis], overlay_image, intersection_image)

        intersection_images.append(final_image)
        
        if save:
            image_to_save = img_as_ubyte(final_image)

            if len(image_to_save.shape) not in [2, 3]:
                raise ValueError(f"Unexpected image shape for saving: {image_to_save.shape}")

            imsave(os.path.join(save_path, f'intersection_image_{idx}.png'), image_to_save)
    
    return intersection_images



In [ ]:
intersection_images = find_intersection_and_save(test_features_, masker_shap_, new_mask, save=True, save_path="/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/Intersection/")

In [ ]:
### Train on new Data

In [ ]:
new_path__ = '/home/maxwellsam/SHAP_LIME_COVID-19/backup_93_perc/dataset_important_features/VGG19/Intersection/'
new_prefix_ = 'ct_covid'

In [ ]:
ct_covid_intersection_features_df =  processImages(new_path__,1)
ct_covid_intersection_features_df.shape

In [ ]:
cvd_imgs_intersection = [ct_noncovid_features_df, ct_covid_intersection_features_df]
cvd_imgs_dataset = pd.concat(cvd_imgs_intersection)
for i in range(100):
    # shuffle the DataFrame rows
    cvd_imgs_dataset_intersection = cvd_imgs_dataset.sample(frac = 1)
# cvd_imgs_dataset_colour = cv2.cvtColor(cvd_imgs_dataset, cv2.COLOR_BGR2RGB)
display(cvd_imgs_dataset_intersection)

In [ ]:
input_data_intersection_x = cvd_imgs_dataset_intersection.iloc[:,:-1].to_numpy().reshape((3103,300,300,1))
#input_data_x = cvd_imgs_dataset.iloc[:,:-1].to_numpy()
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
cvd_imgs_dataset_intersection['output_encode'] = label_encoder.fit_transform(cvd_imgs_dataset_intersection['class_label'])
cvd_imgs_dataset_intersection
cvd_imgs_dataset_intersection = pd.get_dummies(cvd_imgs_dataset_intersection, columns =['output_encode'])
##Getting the input_labels and input_features for training and testing model
output_label_intersection_y = np.array(cvd_imgs_dataset_intersection[['output_encode_0','output_encode_1']])
# print('Input_x Data: \n{0}'.format(input_data_x))
# print('Output_y Data: \n{0}'.format(output_label_y))
print('Input_x Data Shape: \n{0}'.format(input_data_intersection_x.shape))
print('Output_y Data Shape: \n{0}'.format(output_label_intersection_y.shape))

In [ ]:
from sklearn.model_selection import train_test_split

train_features_intersection, test_features_intersection, train_labels_intersection, test_labels_intersection = train_test_split(
    input_data_intersection_x, output_label_intersection_y, test_size=.20, random_state=42)

In [ ]:
modelss = VGG19(input_shape=(300, 300, 1), classes=2)
modelss.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model_vgg19.fit(train_features_intersection, train_labels_intersection, epochs=10, validation_data=(test_features_intersection, test_labels_intersection))